# Clustering techniques

In this notebook you will learn how to perform cluster analysis. We will cover two of the most common clustering methods: hierarchical clustering and k-Means. The content of this notebook is mostly based on the examples from the text book.

> (c) 2019 Galit Shmueli, Peter C. Bruce, Peter Gedeck 
>
> Code included in
>
> _Data Mining for Business Analytics: Concepts, Techniques, and Applications in Python_ (First Edition) 
> Galit Shmueli, Peter C. Bruce, Peter Gedeck, and Nitin R. Patel. 2019.

Let's start by importing all the required libraries:

In [ ]:
import dmba

import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import pairwise
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from pandas.plotting import parallel_coordinates

%matplotlib inline

## Dataset

As always, we start by loading the dataset. We will again load it directly from the `dmba` library. This dataset has information about energy companies in the US.

In [ ]:
df = dmba.load_data('Utilities.csv')
df.head()

Next, we will set the company column as index, which will facilitate further modelling:

In [ ]:
df.set_index('Company', inplace=True)
df.head()

Do you see how all company names are now in bold? We have them as index, which also means that we can easily access particular records using the `.loc` option from `pandas`. 

Now we want to convert all the columns to float variables. This is in principle not necessary, but will avoid a warning from the scale function later on and it will be a good opportunity to practice the usage of for loops:

In [ ]:
# TODO: write a for loop to convert all columns to float variables


Now you saw why for loops are so usefull in programming. You were able to change all column types using 2 lines of code, instead of manually having to change one-by-one, which would take 8 lines of code and be a much less elegant solution. You could, in principle, argue that only 2 columns were not floats, but in practice we have many situations where you would indeed need to run the same operation in multiple columns and this exercise exemplifies how you can do it.

I wanted you to write this for loop, but `pandas` has a method called `apply` that can be used to run a function along rows or columns of a dataframe:

In [ ]:
df = df.apply(lambda x: x.astype('float64'))
df.head()

This solution is even simpler and takes only 1 line of code! The `apply` method is very usefull when you want to perform data transformantions. But what just happened? We basically performed an operation row-wise using the `lambda` construction, which defines an anonymous function. In this case, the input parameter is `x`, which is each row of the dataframe, and the output of the function is all columns of a row converted to float type.


## Distances between points

We saw in the theoretical lesson that we need to know distance between points in order to perform clustering. We also saw that Euclidean distance is the most common method. But do not forget that it is not the only one and not always the most appropriate. In any case, you will now see how to compute the Euclidean distance between all the data points:

In [ ]:
d = pairwise.pairwise_distances(df, metric='euclidean')
pd.DataFrame(d, columns=df.index, index=df.index).head()

So you can see that the distance between Arizona and itself is naturally zero, which is also one of the properties of distances. The distances between Arizona-Boston and Boston-Arizona are both 3989, which is another property of distances. You can also verify that all distances are greater or equal to zero and that the triangle inequality property holds.

But let's now have a look at a different distance measure:

In [ ]:
# TODO: compute the Manhattan distance between the points


In [ ]:
# TODO: what is the Manhattan distance between NY and New England?
# What you learned in the slicing part of the pandas notebook might be usefull here


We saw that it is important to normalize the data before computing distance metrics, as different scales for the variables might largely influence this measure. There are two ways to normalize the input variables. The Pandas standard calculates the sample standard deviation, whereas scikit-learn uses the population standard deviation. The normalized data from the two methods will therefore differ slightly. We will use the Pandas approach:

In [ ]:
# Population standard deviation
df_norm = df.apply(preprocessing.scale, axis=0)

# Sample standard deviation
df_norm = (df - df.mean())/df.std()

df_norm.head()

Let's now have a look at the normalized distances:

In [ ]:
d_norm = pairwise.pairwise_distances(df_norm, metric='euclidean')
pd.DataFrame(d_norm, columns=df.index, index=df.index).head()

Have you noticed something funny? You might now see that the distance between some points and themselves is not quite zero (although very close!). This is due to numerical approximations and the precision of the floats.


## Hierarchical clustering

We will now use the hierarchical agglomerative clustering method to cluster the data. We will use the `linkage` function from the `scipy` library, which is another usefull library for data mining. We saw how to calculate distance between points above, but the algorithm does it for us automatically, it will basically perform all steps you saw in the lecture. All we need to give as input is the normalized data, the linkage method, and the desired distance metric. We will run the algorithm on the data and visualize the results using a dendrogram plot:

In [ ]:
Z = linkage(df_norm, method='single', metric='euclidean')

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(bottom=0.23)
plt.title('Hierarchical Clustering Dendrogram (Single linkage)')
plt.xlabel('Company')
dendrogram(Z, labels=list(df_norm.index), color_threshold=2.75)
plt.axhline(y=2.75, color='black', linewidth=0.5, linestyle='dashed')
plt.show()

In [ ]:
# TODO: run hierarchical clustering using average linkage and Euclidean distance
# Plot the result of the algorithm as a dendrogram and use a cutoff of 3.6


Let's now visualize the distribution of the input variables, which can help in understanding the clusters:

In [ ]:
# Get clusters for average linkage
avg_linkage = fcluster(linkage(df_norm, 'average'), 6, criterion='maxclust')
avg_linkage = pd.Series(avg_linkage, index=df_norm.index)

# Visualize distributions of numerical variables
df_vis = df_norm.copy(deep=True)
df_vis.index = ['{}: {}'.format(cluster, state) for cluster, state in zip(avg_linkage, df_vis.index)]
sns.clustermap(df_vis, method='average', col_cluster=False,  cmap='mako_r')
plt.show()

We can, for instance, see that the cluster containing Nevada, Idaho and Puget has high sales and low nuclear energy fraction. The cluster that contains Madison, on the other hand, has high nuclear energy fraction and low sales. We can try to make sense of the groups and interpret the clusters in such fashion. There is no formal way to validate clusters, as we saw in the lectures, therefore interpretation and usability is key.


## K-Means

Let's now see how to use k-Means, which is the second method you learned about. The only required input parameter is `k`, which defines the number of clusters. So let's start with `k=6`, which is what we chose from hierarchical clustering.

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(df_norm)

It is as simple as that (for you, as Python is doing all the hard work ;))! The algorithm will randomly choose 6 initial centroids, here again `random_state` controls reproducibility, and will keep updating the clusters until convergence is achieved.

Let's now have a look at the result:

In [ ]:
# Cluster membership
kmeans6 = pd.Series(kmeans.labels_, index=df_norm.index)
for key, item in kmeans6.groupby(kmeans6):
    print(key, ': ', ', '.join(item.index))

The results of k-Means are good to compare with hierarchical clustering using average linkage. Since both tend to produce spherical-like clusters where dispersion is minimized. So let's see again the previous clusters:

In [ ]:
# TODO: print cluster membership for the average linkage method


We can see that the results are not exactly the same, but are very compatible.

Let's now organize the results of k-Means and see another way of interpreting results. We will start by making a dataframe with the clusters centroids:

In [ ]:
df_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=df_norm.columns)
df_centroids['cluster'] = ['Cluster {}'.format(i) for i in df_centroids.index]
df_centroids

You saw above that you can use for loops inside lists to help you with list creation, that is quite a Pythonic solution.

Let's now use a parallel coordinate plot to understand the properties of each cluster:

In [ ]:
plt.figure(figsize=(10,6))
fig.subplots_adjust(right=3)
ax = parallel_coordinates(df_centroids, class_column='cluster', colormap='Dark2', linewidth=5)
plt.legend(loc='center left', bbox_to_anchor=(0.95, 0.5))
plt.xlim(-0.5,7.5)
plt.show()

First of all, we see here again the same pattern of the two clusters with low sales and high nuclear energy and the reversed pattern, which is in line with the previous interpretation. We also see that the NY cluster is the one with the lowest sales and highest fixed charge, whereas the Nevada cluster has the right opposite behaviour. Again you can use such visualization for cluster interpretation. They can also be quite usefull when searching for a label for each cluster.


## Elbow method

What if we have no idea what the best number of clusters is? As we discussed in the theoretical lesson, we can use the Elbow method for choosing `k`. Let's see how to do it. Here again a for loop can help us. We will loop through different `k`'s and keep the average within cluster squared distances, which is a measure of dispersion:

In [ ]:
inertia = []
for n_clusters in range(1, 10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df_norm)
    inertia.append(kmeans.inertia_ / n_clusters)

Let's now organize our results into a dataframe:

In [ ]:
df_inertias = pd.DataFrame({'n_clusters': range(1, 10), 'inertia': inertia})

Finally, we can visualize the results:

In [ ]:
ax = df_inertias.plot(x='n_clusters', y='inertia')
plt.xlabel('Number of clusters(k)')
plt.ylabel('Average Within-Cluster Squared Distances')
plt.ylim((0, 1.1 * df_inertias.inertia.max()))
ax.legend().set_visible(False)
plt.show()

We can see that the elbow is around `k=3` (some might argue 2). So if you knew nothing, you could start here and see whether the clusters are usefull and meaningfull. 


### Homework

If you are curious, you can go on and see what happens if you choose k=3 for k-Means. So I will leave that as homework or for if we have spare time.

In [ ]:
# TODO: run k-Means for k=3 and check the cluster membership


In [ ]:
# TODO: visualize the parallel coordinate plot for the results of k-Means with k=3


In [ ]:
# TODO: check the distributions of each variable for these 3 clusters
# The heatmap option from the seaborn library is a good choice, we used it in Lab 2
# This exercise might need some extra originality, but you have all the tools to solve it


Which label would you give for these clusters? Are the clusters usefull and understandable?